In [44]:
import os, pprint

template_api_path = "../sample_files/base-template/"
source_api_path = "../sample_files/api-current/"
destination_api_path = "../sample_files/api-destination/"

filename = "build.gradle"
template_file = os.path.join(template_api_path, filename)
source_file = os.path.join(source_api_path, filename)
destination_file = os.path.join(destination_api_path, filename)

In [45]:
# with open(template_file) as file:
#     for line in file:
#         print(line.rstrip())

In [46]:
with open(template_file) as f:
    line_start = 0
    line_end = 0
    for index, line in enumerate(f):
        if line.startswith("dependencies {"):
            line_start = index
        if line_start !=0 and line.startswith("}"):
            line_end = index
            break
            
with open(template_file) as f:
    i = 0
    template_list = []
    default_list = []
    for index, line in enumerate(f):
        if index > line_start and index < line_end and line !='\n':
            default_list.append(line)
            rep_dic = {}
            tmp_line = line.strip().replace("'", "").replace("ion ", "ion:").replace(" ", "")
#             print("Line {}: {}".format(index, tmp_line))
            i += 1
            if i == 1 or i == 8 or i >= 13:
                temp = tmp_line.split(":")
                if len(temp) > 1:
                    ver_list = temp[len(temp)-1].split('.')
                    ver_list = [int(x) for x in ver_list]
                    
                    rep_dic["prefix"] = temp[0]
                    rep_dic["artifact"] = temp[len(temp)-2]
                    rep_dic["version"] = temp[len(temp)-1]
                    rep_dic["version_sum"] = sum(ver_list)
                    rep_dic['template_line'] = line
                    rep_dic['rule'] = 1
            elif i == 10:
                temp = tmp_line.replace(",", ":").split(":")
                if len(temp) > 1:
                    ver_list = temp[len(temp)-1].split('.')
                    ver_list = [int(x) for x in ver_list]
                    
                    rep_dic["prefix"] = temp[0]
                    rep_dic["artifact"] = temp[len(temp)-3]
                    rep_dic["version"] = temp[len(temp)-1]
                    rep_dic["version_sum"] = sum(ver_list)
                    rep_dic['template_line'] = line
                    rep_dic['rule'] = 2
            else:
                temp = tmp_line.split(":")
                if len(temp) > 1:
                    rep_dic["prefix"] = temp[0]
                    rep_dic["artifact"] = temp[len(temp)-1]
                    rep_dic["version"] = None
                    rep_dic["version_sum"] = 0
                    rep_dic['template_line'] = line
                    rep_dic['rule'] = None
            template_list.append(rep_dic)
                
# pprint.pprint(template_list)
pprint.pprint(default_list)

["\timplementation 'com.barclays.api:api-core-boot:2.0.21'\n",
 "\timplementation 'org.springframework.cloud:spring-cloud-config-server'\n",
 "\timplementation 'org.springframework.boot:spring-boot-starter-thymeleaf'\n",
 '\timplementation '
 "'org.springframework.cloud:spring-cloud-starter-vault-config'\n",
 "\timplementation 'org.springframework.cloud:spring-cloud-starter-bus-amqp'\n",
 "\timplementation 'org.springframework.cloud:spring-cloud-config-monitor'\n",
 '\timplementation '
 "'org.springframework.cloud:spring-cloud-starter-netflix-eureka-client'\n",
 "\timplementation 'org.bitbucket.b_c:jose4j:0.7.3'\n",
 "\ttestImplementation 'org.springframework.boot:spring-boot-starter-test'\n",
 "\ttestImplementation group: 'com.github.tomakehurst', name: 'wiremock', "
 "version: '2.26.3'\n",
 "\ttestImplementation 'junit:junit'\n",
 "\ttestImplementation 'org.mockito:mockito-core'\n",
 "\ttestImplementation 'org.raml.raml-parser-2:1.0.13'\n",
 "\ttestImplementation 'com.intuit.karate:k

In [70]:
from ordered_set import OrderedSet

with open(template_file, "r") as f, open(source_file, "r+") as out:
    line_start = 0
    line_end = 0
    for index, line in enumerate(f):
        if line.startswith("dependencies {"):
            line_start = index
        if line_start !=0 and line.startswith("}"):
            line_end = index
            break

    line_start_out = 0
    line_end_out = 0
    for index_out, line_out in enumerate(out):
        if line_out.startswith("dependencies {"):
            line_start_out = index_out
        if line_start_out !=0 and line_out.startswith("}"):
            line_end_out = index_out
            break

with open(template_file) as f, open(source_file, "r") as out:
    for index, line in enumerate(f):
        if index > line_start and index < line_end:
            temp_in = line.strip().replace("'", "").replace("tion ", "tion:").replace(" ", "").split(":")
#             print("Line {}: {}".format(index, line.strip()))
#             print("Line {}: {}".format(index, temp_in))
            
#     print("\n*******============*******\n")
    
    template_set = OrderedSet()
    unchanged_set = OrderedSet()
    changed_set = OrderedSet()
    existing_file_list = []
    for index_out, line_out in enumerate(out):
        if index_out > line_start_out and index_out < line_end_out:
            existing_file_list.append(line_out)
#             print("Line {}: {}".format(index_out, line_out.strip()))
            for l in template_list:
                if l['artifact'] in line_out:
                    template_set.add(l['template_line'])
                    changed_set.add(line_out)
                else:
                    unchanged_set.add(line_out)

#     pprint.pprint(existing_file_list)
#     pprint.pprint(unchanged_set)
#     pprint.pprint(unchanged_set)

    diff = (unchanged_set - changed_set)

#     pprint.pprint(diff)
    t = []
    t.append(['\n\ndependencies {\n'])
    t.append(template_set)
    t.append(diff)
    t.append(['}\n'])
    print("\n*******============*******\n")
    dep_list = [i for sub in t for i in sub]
#     print(dep_list)
    print("\n*******============*******\n")
#     for row in dep_list:
#         print(row)

        
    words = [w.replace('compile', 'implementation') for w in dep_list]
    pprint.pprint(words)

with open(source_file, "r") as out:
    current_list = []
    for index_, line_ in enumerate(out):
        if index_ > line_start_out-1 and index_ < line_end_out+1:
            continue
        current_list.append(line_)
        


for row in words:
    current_list.append(row)

# pprint.pprint(current_list)


*******============*******


*******============*******

['\n\ndependencies {\n',
 "\timplementation 'com.barclays.api:api-core-boot:2.0.21'\n",
 "\timplementation 'org.springframework.cloud:spring-cloud-config-server'\n",
 "\timplementation 'org.springframework.boot:spring-boot-starter-thymeleaf'\n",
 '\timplementation '
 "'org.springframework.cloud:spring-cloud-starter-vault-config'\n",
 "\timplementation 'org.springframework.cloud:spring-cloud-starter-bus-amqp'\n",
 "\timplementation 'org.springframework.cloud:spring-cloud-config-monitor'\n",
 '\timplementation '
 "'org.springframework.cloud:spring-cloud-starter-netflix-eureka-client'\n",
 "\timplementation 'org.bitbucket.b_c:jose4j:0.7.3'\n",
 "\ttestImplementation 'org.springframework.boot:spring-boot-starter-test'\n",
 "\ttestImplementation group: 'com.github.tomakehurst', name: 'wiremock', "
 "version: '2.26.3'\n",
 "\ttestImplementation 'junit:junit'\n",
 "\ttestImplementation 'org.mockito:mockito-core'\n",
 "\ttestImplementat

In [71]:
with open(destination_file, "w+") as out:

    for line in current_list:
        out.writelines(line)